# Feature extraction

Here we will add manually extracted features to the cleaned, resampled and merged table.

This code will get abstracted into `src/house_climate/data/extract_features.py`

**Prerequisites:**

Execute the below to prepare dataset 

```bash
make data_interim
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from dotenv import find_dotenv

In [3]:
REPO_ROOT = Path(find_dotenv()).parent
CLEANED_DATA = REPO_ROOT / "data" / "interim" / "01_preprocess.parquet"

___

## Load in preprocessed parquet file

In [4]:
merged = pd.read_parquet(CLEANED_DATA)
merged

,humidity,temperature,zone_id,value,weather,exterior_temp,zone_type,zone_name,hours_in_day
timestamp,,,,,,,,,
2021-12-31 23:45:00+00:00,0.500,20.00,1,NONE,UNKNOWN,NaN,HEATING,Living room,24.0
2021-12-31 23:45:00+00:00,0.500,20.00,2,NONE,UNKNOWN,NaN,HEATING,Bedroom,24.0
2021-12-31 23:45:00+00:00,0.500,20.00,6,NONE,UNKNOWN,NaN,HEATING,Haadiyah Room,24.0
2021-12-31 23:45:00+00:00,0.500,20.00,7,NONE,UNKNOWN,NaN,HEATING,Ensuite,24.0
2021-12-31 23:45:00+00:00,0.500,20.00,9,NONE,UNKNOWN,NaN,HEATING,Office,24.0
...,...,...,...,...,...,...,...,...,...
2023-01-02 00:15:00+00:00,0.557,19.87,9,HIGH,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0
2023-01-02 00:15:00+00:00,0.585,20.06,10,NONE,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0
2023-01-02 00:15:00+00:00,0.579,20.35,12,NONE,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0


In [5]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4190470 entries, 2021-12-31 23:45:00+00:00 to 2023-01-02 00:15:00+00:00
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   humidity       float64
 1   temperature    float64
 2   zone_id        Int8   
 3   value          object 
 4   weather        object 
 5   exterior_temp  float64
 6   zone_type      object 
 7   zone_name      object 
 8   hours_in_day   float64
dtypes: Int8(1), float64(4), object(4)
memory usage: 295.7+ MB


In [6]:
merged.describe()

,humidity,temperature,zone_id,exterior_temp,hours_in_day
count,4.190470e+06,4.190470e+06,4190470.0,13101.000000,4.190470e+06
mean,5.905803e-01,2.138312e+01,7.884678,1.989455,2.400069e+01
std,7.457257e-02,1.615118e+00,4.755862,0.631705,7.414660e-02
min,2.760000e-01,1.292000e+01,1.0,0.400000,2.300000e+01
25%,5.320000e-01,2.029000e+01,2.0,1.620000,2.400000e+01
50%,5.810000e-01,2.144000e+01,7.0,1.900000,2.400000e+01
75%,6.406667e-01,2.239933e+01,10.0,2.470000,2.400000e+01
max,9.010000e-01,2.943000e+01,17.0,3.330000,2.500000e+01


___

## Add features

In [7]:
def add_features(df):
    df = (
        df
        .assign(hour_of_day=lambda x: x.index.hour,
                day_of_week=lambda x: x.index.weekday,
                day_of_year=lambda x: x.index.dayofyear,
                )
        .assign(day_night=lambda x: pd.cut(x.hour_of_day, bins=[0,8,20], labels=['night', 'day']),
                is_weekend=lambda x: x.day_of_week.isin([5, 6]),
                )
    )
    return df

with_features = merged.pipe(add_features)
with_features

,humidity,temperature,zone_id,value,weather,exterior_temp,zone_type,zone_name,hours_in_day,hour_of_day,day_of_week,day_of_year,day_night,is_weekend
timestamp,,,,,,,,,,,,,,
2021-12-31 23:45:00+00:00,0.500,20.00,1,NONE,UNKNOWN,NaN,HEATING,Living room,24.0,23,4,365,NaN,False
2021-12-31 23:45:00+00:00,0.500,20.00,2,NONE,UNKNOWN,NaN,HEATING,Bedroom,24.0,23,4,365,NaN,False
2021-12-31 23:45:00+00:00,0.500,20.00,6,NONE,UNKNOWN,NaN,HEATING,Haadiyah Room,24.0,23,4,365,NaN,False
2021-12-31 23:45:00+00:00,0.500,20.00,7,NONE,UNKNOWN,NaN,HEATING,Ensuite,24.0,23,4,365,NaN,False
2021-12-31 23:45:00+00:00,0.500,20.00,9,NONE,UNKNOWN,NaN,HEATING,Office,24.0,23,4,365,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-02 00:15:00+00:00,0.557,19.87,9,HIGH,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0,0,0,2,NaN,False
2023-01-02 00:15:00+00:00,0.585,20.06,10,NONE,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0,0,0,2,NaN,False
2023-01-02 00:15:00+00:00,0.579,20.35,12,NONE,NIGHT_CLOUDY,0.4,HEATING,Upstairs bathroom,24.0,0,0,2,NaN,False


In [8]:
with_features.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4190470 entries, 2021-12-31 23:45:00+00:00 to 2023-01-02 00:15:00+00:00
Data columns (total 14 columns):
 #   Column         Dtype   
---  ------         -----   
 0   humidity       float64 
 1   temperature    float64 
 2   zone_id        Int8    
 3   value          object  
 4   weather        object  
 5   exterior_temp  float64 
 6   zone_type      object  
 7   zone_name      object  
 8   hours_in_day   float64 
 9   hour_of_day    int64   
 10  day_of_week    int64   
 11  day_of_year    int64   
 12  day_night      category
 13  is_weekend     bool    
dtypes: Int8(1), bool(1), category(1), float64(4), int64(3), object(4)
memory usage: 399.6+ MB


In [9]:
with_features.describe()

,humidity,temperature,zone_id,exterior_temp,hours_in_day,hour_of_day,day_of_week,day_of_year
count,4.190470e+06,4.190470e+06,4190470.0,13101.000000,4.190470e+06,4.190470e+06,4.190470e+06,4.190470e+06
mean,5.905803e-01,2.138312e+01,7.884678,1.989455,2.400069e+01,1.150023e+01,3.013901e+00,1.938560e+02
std,7.457257e-02,1.615118e+00,4.755862,0.631705,7.414660e-02,6.922289e+00,2.002110e+00,1.054759e+02
min,2.760000e-01,1.292000e+01,1.0,0.400000,2.300000e+01,0.000000e+00,0.000000e+00,1.000000e+00
25%,5.320000e-01,2.029000e+01,2.0,1.620000,2.400000e+01,6.000000e+00,1.000000e+00,1.030000e+02
50%,5.810000e-01,2.144000e+01,7.0,1.900000,2.400000e+01,1.200000e+01,3.000000e+00,2.040000e+02
75%,6.406667e-01,2.239933e+01,10.0,2.470000,2.400000e+01,1.800000e+01,5.000000e+00,2.850000e+02
max,9.010000e-01,2.943000e+01,17.0,3.330000,2.500000e+01,2.300000e+01,6.000000e+00,3.650000e+02


Dataset saved to disk by the python module version of this feature extraction pipeline